###### References: 
- https://docs.python.org/3/tutorial/datastructures.html   
- Fluent Python by Luciano Ramalho. Chapter 4: Text versus Bytes


# Character Issues
## Unicode 
The Unicode standard separates the identity of characters from specific byte representations.
* The character id, i.e. _code point_, is a number from 0 to 1,114,111 in the Unicode standdard
* U+ prefix
* The actual bytes depend on the _encoding_, e.g. `A` (U+0041) is encoded as a single byte in UTF-8, `\x41`, or as `\x41\x00` in UTF-16LE  encoding.

#### Encoding and decoding:

In [ ]:
s = 'cafe'
len(s)

In [ ]:
b = s.encode('utf8')
b

In [ ]:
len(b)

In [ ]:
b.decode('utf8')

# Byte Essentials
## `bytes` or `bytearray`
Each item is an integer from 0 to 255, and not a one character string.

 `bytes` object is immutable; `bytearray` object allows you to modify its elements.
#### A five-byte sequence as  bytes and bytearray

In [ ]:
cafe = bytes('cafe', encoding='utf-8')
cafe

In [ ]:
cafe[0]

In [ ]:
cafe[:1]

In [ ]:
cafe_arr = bytearray(cafe)
cafe_arr

In [ ]:
cafe_arr[1:]

### Building bytes and  bytearray
#### `fromhex`

In [ ]:
bytes.fromhex('31 48 CE A9')

* using a `str` and an `encoding`
* An iterable providing items with values from 0 to 255
* An object that implements the buffer protocol  (e.g. `bytes`, `bytearray`, `memoryview`, `array.array`);  this copies the bytes from the source object to the newly created binary sequence
####  Initializing bytes from the raw data of  an array

In [ ]:
import array

numbers = array.array('h', [-2, -1, 0, 1, 2])
octets = bytes(numbers)
octets

Creating a `bytes` or `bytearray` from any buffer-like source will always copy the bytes. In contract, `memoryview` objects let you share memory between binary data structures.
# Structs and Memory Views
`struct` module provides functions to parse packed bytes into a tuple of fields of different types of different types and to perform the opposite conversion.

`memoryview` class does not let you create or store byte sequences, but provides a shared memory access to   slices of data from other binary sequnences, etc, without copying the bytes.

####  Using  `memoryview` and `struct` to inspect a GIF image header

In [ ]:
import struct
fmt = '<3s3sHH' # struct format: < little endian;3s3s two sequences of 3 bytes; HH two 16-bit integers;

with open('20220219/women_who_code.gif', 'rb') as fp:
    img = memoryview(fp.read())
    
header = img[:10]
bytes(header)

In [ ]:
struct.unpack(fmt, header) # type, version, width, height

In [ ]:
del header
del img

##### Ref: https://docs.python.org/3/library/mmap.html

# Basic Encoders / Decoders

The Python distribution bundles more than 100 _codecs_ for text to byte conversion and vice versa.

In [ ]:
for codec in ['latin_1', 'utf_8', 'utf_16']:
    print(codec, "El Niño".encode(codec), sep='\t')

## Understanding Encode/Decode Problems
###  Coping with UnicodeEncodeError

In [ ]:
city = "São Paulo"
city.encode('utf_8')

In [ ]:
city.encode('utf_16')

In [ ]:
city.encode('iso8859_1')  # "Latin alphabet no. 1"

In [ ]:
city.encode('cp437')  # character set of the original IBM PC 

In [ ]:
city.encode('cp437', errors='ignore')

In [ ]:
city.encode('cp437', errors='replace')

In [ ]:
city.encode('cp437', errors='xmlcharrefreplace')

###  Coping with UnicodeDecodeError

In [ ]:
octets = b'Monter\xe9al'
octets.decode('cp1252')  # Windows-1252 is a single-byte character encoding of the Latin alphabet

In [ ]:
octets.decode('iso8859_7')  # Part 7: Latin/Greek alphabet

In [ ]:
octets.decode('koi8_r')

In [ ]:
octets.decode('utf_8')

In [ ]:
octets.decode('utf_8', errors='replace')

### SyntaxError

In [ ]:
%run 20220219/hello.py

## Discover Encoding
### Chardet
Character Encodding Detector

https://pypi.python.org/pypi/chardet

### BOM
byte-order-mark

UTF-16 encodings prepends text to be encoded with `ZERO WIDTH NO-BREAK SPACE` U+FEFF.

Such that little-endian system is encoded as `'\xff\xfe'`

In [ ]:
u16 = 'Olá Mundo'.encode('utf_16')
u16

In [ ]:
u16le = 'Olá Mundo'.encode('utf_16le')
list(u16le)

In [ ]:
u16be = 'Olá Mundo'.encode('utf_16be')
list(u16be)

# Handling Text Files
##  The Unicode Sandwich
* Decode bytes on input
 *  process text only
* Encode text on output

In [ ]:
open('cafe.txt', 'w', encoding='utf_8').write('café')

In [ ]:
open('cafe.txt', encoding='cp1252').read()

In [ ]:
fp = open('cafe.txt')
fp

In [ ]:
fp.close()

Write size and encoded file size

In [ ]:
import os

os.stat('cafe.txt').st_size

In [ ]:
fp2 = open('cafe.txt')
fp2

In [ ]:
fp2.encoding

reading in binary

In [ ]:
fp3 = open('cafe.txt', 'rb')
fp3

In [ ]:
fp3.read()

## Encoding defaults

In [ ]:
import sys, locale

In [ ]:
expressions  = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
        """

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))
    

# Normalizing Unicode for saner comparison

In [ ]:
s1 = 'café'
s2 = 'cafe\u0301'
s1, s2

In [ ]:
len(s1), len(s2)

In [ ]:
from unicodedata import normalize

In [ ]:
len(normalize('NFC', s1)), len(normalize('NFC', s2))

In [ ]:
len(normalize('NFD', s1)), len(normalize('NFD', s2))

In [ ]:
normalize('NFC', s1) == normalize('NFC', s2)

In [ ]:
normalize('NFD', s1) == normalize('NFD', s2)

Certain characters are similar, e.g.:

In [ ]:
from unicodedata import name

In [ ]:
ohm = '\u2126'

In [ ]:
name(ohm)

In [ ]:
ohm_c  = normalize('NFC', ohm)
name(ohm_c)

In [ ]:
ohm == ohm_c

In [ ]:
normalize('NFC', ohm) == normalize('NFC', ohm_c)

### Compatibility in characters
NFKC  , NFKD

In [ ]:
half =  '½'
normalize('NFKC', half)

In [ ]:
four_squared = '4²'
normalize('NFKC', four_squared)

In [ ]:
micro = 'µ'
micro_kc = normalize('NFKD', micro)
micro, micro_kc

In [ ]:
ord(micro), ord(micro_kc)

In [ ]:
name(micro), name(micro_kc)

Normalise with care. In some cases, it is useful for indexing and searching.

## Case folding
`s.casefold()` is converting all text to lowercase, with some additional transformation.

In [ ]:
name(micro)

In [ ]:
micro_cf = micro.casefold()
name(micro_cf)

In [ ]:
micro, micro_cf

In [ ]:
eszett = 'ß'
name(eszett)

In [ ]:
eszett_cf = eszett.casefold()

In [ ]:
eszett, eszett_cf

## Utility Functions for Normalized Text Matching
For most cases, `str.casefold()` is good for case-insensitive comparison.

If you work with text in many languages, creating a pair of functions like `nfc_equal` and `fold_equal` are useful.

In [ ]:
def nfc_equal(str1, str2):
    return normalize('NFC', str1) == normalize('NFC', str2)

def fold_equal(str1, str2):
    return (normalize('NFC', str1).casefold() ==
            normalize('NFC', str2).casefold())

In [ ]:
s1, s2

In [ ]:
s1 == s2

In [ ]:
nfc_equal(s1,  s2)

In [ ]:
nfc_equal('a', 'A')

In [ ]:
fold_equal('a', 'A')

In [ ]:
street1 = 'Straße'
street2 = 'strasse'
street1 == street2

In [ ]:
nfc_equal(street1, street2)

In [ ]:
fold_equal(street1, street2)

## Extreme "Normalization":  Take  out  Diacritics
Diacritics are __marks placed above or below__ (or sometimes next to) a letter in a word to indicate a particular pronunciation—in regard to accent, tone, or stress—as well as meaning

Google search ignores diacritics, (e.g. accents, cedillas, etc.), in certain contexts.

In [ ]:
import unicodedata
import string

def shave_marks(txt):
    """Remove all diacritic marks"""
    norm_text = unicodedata.normalize('NFD', txt)  # Decompose characters into base char and combining marks
    shaved = ''.join(c for c  in norm_text if not unicodedata.combining(c)) # Filter out combining marks
    return unicodedata.normalize('NFC', shaved) # Recompose all characters

In [ ]:
order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'
shave_marks(order)

In [ ]:
greek = 'Ζέφυρος, Zéfiro'
shave_marks(greek)

In [ ]:
def shave_marks_latin(txt):
    """Remove all diacritic marks from Latin base characters"""
    norm_txt = unicodedata.normalize('NFD', txt)  
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:   # Skip over combining marks when base char is Latin
            continue  # ignore diacritic on Latin base char
        keepers.append(c)                             # Otherwise keep current char
        # if it isn't combining char, it's a new base char
        if not unicodedata.combining(c):              # Detect new base char and dertermine if it's Latin
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)   # Recompose all characters

In [ ]:
shave_marks_latin(greek)

In [ ]:
single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",  # Build mapping table for char-to-char replacement
                           """'f"*^<''""---~>""")

multi_map = str.maketrans({  # Build mapping table for char-to-string replacement
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)  # Merge mapping tables


def dewinize(txt):
    """Replace Win1252 symbols with ASCII chars or sequences"""
    return txt.translate(multi_map)  # does not affect ASCII or latin1 text


def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))     # Remove diacritical marks
    no_marks = no_marks.replace('ß', 'ss')          # We want to preserve the case
    return unicodedata.normalize('NFKC', no_marks)  # Recompose with compatibility codepoints

In [ ]:
shave_marks_latin(order)

In [ ]:
dewinize(order)

In [ ]:
asciize(order)

#  Sorting Unicode Text

In [ ]:
fruits = ['Cupuaçu', 'Açaí', 'Acerola', 'Cajá', 'Caju']
sorted(fruits)

In [ ]:
import locale

In [ ]:
locale.setlocale(locale.LC_COLLATE,  'pt_BR.UTF-8')

In [ ]:
sorted(fruits, key=locale.strxfrm)

## Sorting with Unicode Collation Algorithm

In [ ]:
import pyuca

In [ ]:
coll = pyuca.Collator()

In [ ]:
sorted(fruits, key=coll.sort_key)

# The Unicode Database

In [ ]:
import unicodedata
import re

In [ ]:
re_digit = re.compile(r'\d')

sample = '1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

In [ ]:
for char in sample:
    print('U+%04x' % ord(char),                       # Code point in U+0000 format
          char.center(6),                             # Characterized in str len of 6
          're_dig' if re_digit.match(char) else '-',  # Show re_dig if character matches the regex
          'isdig' if char.isdigit() else '-',         # Show isdig if char.isdigit()
          'isnum' if char.isnumeric() else '-',       # Show isnum if char.isnumeric()
          format(unicodedata.numeric(char), '5.2f'),  # Numeric value formated, width 5 and 2 decimal place
          unicodedata.name(char),                     # Unicode character name
          sep='\t')

# Dual-Mode str and bytes APIs
## `str` Versus `bytes` in Regular Expressions
Hardy–Ramanujan number 1729 : the smallest number expressible as the sum of two cubes in two different ways.

In [ ]:
re_numbers_str = re.compile(r'\d+')     # String types
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')  # Byte types
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"  # Unicodde text to search, contining Tamil digits 1729
            " as 1729 = 1³ + 12³ = 9³ + 10³.")        # String literal concatenation

text_bytes = text_str.encode('utf_8')  # bytes string is needed to search with the bytes regular expression

print('Text', repr(text_str), sep='\n  ')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))      # The str pattern r'\d+' matches the Tamil and ASCII
print('  bytes:', re_numbers_bytes.findall(text_bytes))  # The bytes pattern rb'\d+' matches only ASCII digits
print('Words')
print('  str  :', re_words_str.findall(text_str))        # The str pattern r'\w+' matches letters, superscripts, Tamil, and ASCII digits.
print('  bytes:', re_words_bytes.findall(text_bytes))    # The bytes pattern rb'\w+' matches only ASCII bytes for letters and  digits.

Regular expression can be used on `str` as well as `bytes`.

`re` on `bytes` outside of the ASCII range are treated as nondigits and nonword characters.

## `str` Versus `bytes` on `os` Functions

`listdir` with `str` and `bytes` arguments and results:

In [ ]:
os.listdir('20220219/')

In [ ]:
os.listdir(b'20220219/')